# DBAAS Adobe Data Model

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM

<dependencies>
  <dependency>
    <groupId>io.netty</groupId>
    <artifactId>netty-transport</artifactId>
     <version>4.1.93.Final</version>
  </dependency>
 
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-proxy-client</artifactId>
    <version>7.0.0</version>
  </dependency>
  
</dependencies>

# Test

In [3]:
import com.aerospike.client.proxy.AerospikeClientProxy;
import com.aerospike.client.policy.*;
import com.aerospike.client.*;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

//Required Imports
//import com.aerospike.client.AerospikeClient;
//import com.aerospike.client.policy.WritePolicy;
//import com.aerospike.client.Bin;
//import com.aerospike.client.Key;
//import com.aerospike.client.Record;
//import com.aerospike.client.Value;
//import com.aerospike.client.Operation;
import com.aerospike.client.query.Statement;
import com.aerospike.client.exp.Exp;

//Required Imports

import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.ListReturnType;

import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.cdt.MapReturnType;

import com.aerospike.client.Operation;
import com.aerospike.client.cdt.MapOperation;

import com.aerospike.client.policy.BatchPolicy;

import java.util.Random;
import java.util.stream.*;

final String AEROSPIKE_CLOUD_HOST="0a3326a2-86f1-4fc9-acb8-b47bbe20fed0.aerospikedev.com";
final String AEROSPIKE_CLOUD_KEY="d2073c5a754948199713b37e970e75b2"; 
final String AEROSPIKE_CLOUD_SECRET="525573c78a7c4f9d9e883dc04d2bd7fc";
final String AEROSPIKE_CLOUD_NAMESPACE="aerospike_cloud";
final int AEROSPIKE_CLOUD_PORT=4000;

ClientPolicy policy = new ClientPolicy();
policy.user = AEROSPIKE_CLOUD_KEY;
policy.password = AEROSPIKE_CLOUD_SECRET;

TlsPolicy tlsPolicy = new TlsPolicy();
policy.tlsPolicy = tlsPolicy;

AerospikeClientProxy client = new AerospikeClientProxy(policy, 
           new Host(AEROSPIKE_CLOUD_HOST, AEROSPIKE_CLOUD_PORT) 
                );

Key key = new Key(AEROSPIKE_CLOUD_NAMESPACE, "my_set", "my_key");
Bin bin = new Bin("my_bin", "Hello World!");

client.put(null, key, bin);
Record record = client.get(null, key);
System.out.println(record);

//client.close();


(gen:2),(exp:0),(bins:(my_bin:Hello World!))


In [4]:
class Truncate {
  public void truncate(String ns, String setName, int sinceUpdate_ms) {
    WritePolicy wPolicy = new WritePolicy();
    if(sinceUpdate_ms > 0){
      //Add "Since Last Update Time" based Expression Filter
      wPolicy.filterExp = Exp.build(Exp.gt(Exp.sinceUpdate(), Exp.val(sinceUpdate_ms)));
    }
    
    Operation[] ops = new Operation[] {
      Operation.delete()
    };
    Statement stmt = new Statement();
    stmt.setNamespace(ns);
    if(setName != null){
      stmt.setSetName(setName);
    }
    stmt.setOperations(ops);
    client.execute(wPolicy, stmt);
  }
}

In [5]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, null, 0);

In [6]:
public static String hex(int n) {
    // call toUpperCase() if that's required
    return String.format("0x%8s", Integer.toHexString(n)).replace(' ', '0').toUpperCase();
}

In [7]:
//Create SegmentId records
class segmentId {
    
  //Use this if only adding ProfileIDs to segment records (Proposed Data Model) 
  public void addProfile (int segmentId, int profileId, int timeStamp, boolean bCheck) {
   
    int keyVal = segmentId;
    keyVal = keyVal << 8;  //Changed to 8 bits ... for ease of reading hex numbers for code dev
    int profileLSBits = profileId & 0xFF  ; // 0x  1111 1111
    keyVal = keyVal | profileLSBits;
      
    //We are choosing 10bits (xx..changed to 8 bits) here, ie 256 records for each segmentId. 
    //We can shard based on actual data size to fewer or higher bits.
    //If profileID is a String, its hash can be used to get LSBs to 
    //to pick which PK=segemtID:LSB record it will go to.
      
    if(bCheck){  //For code dev only, remove later.
      System.out.println("SegmentId:"+ hex(segmentId));
      System.out.println("ProfileId:"+ hex(profileId));
      System.out.println("SegKey:"+ hex(keyVal));
      //System.out.println("SegmentId:"+ Integer.toBinaryString(segmentId));
      //System.out.println("ProfileId:"+ Integer.toBinaryString(profileId));
      System.out.println("SegKey:"+ Integer.toBinaryString(keyVal));
    }
    
    Key segKey = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal); 
    //Bin: In MapOperation, just pass bin name = "ProfileMap"
    WritePolicy wPolicy2 = new WritePolicy();
    wPolicy2.totalTimeout = 10000;  //need larger totalTimeout if writing in a loop on the proxy server.
      
    MapPolicy mPolicy = new MapPolicy(MapOrder.KEY_ORDERED, MapWriteFlags.DEFAULT); 
    
    client.operate(wPolicy2, segKey, 
                   MapOperation.put(mPolicy,"ProfileMap", Value.get(profileId), Value.get(timeStamp)) 
                   );
   
    if(bCheck){
      System.out.println(client.get(null, segKey));
    }   
  }
    
    //Remove a profile
    public void removeProfile (int segmentId, int profileId, boolean bCheck) {
   
    int keyVal = segmentId;
    keyVal = keyVal << 8;
    int profileLSBits = profileId & 0xFF  ; // 0x 1111 1111
    keyVal = keyVal | profileLSBits;
      
    //We are choosing 8 bits here, ie 256 records for each segmentId. 
    //We can shard based on actual data size to fewer or higher bits.
    //If profileID is a String, its hash can be used to get LSBs to 
    //to pick which PK=segemtID:LSB record it will go to.
      
    if(bCheck){  //For code dev only, remove later.
      System.out.println("SegmentId:"+ hex(segmentId));
      System.out.println("ProfileId:"+ hex(profileId));
      System.out.println("SegKey:"+ hex(keyVal));
      //System.out.println("SegmentId:"+ Integer.toBinaryString(segmentId));
      //System.out.println("ProfileId:"+ Integer.toBinaryString(profileId));
      System.out.println("SegKey:"+ Integer.toBinaryString(keyVal));
    }
    
    Key segKey = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal); 
    //Bin: In MapOperation, just pass bin name = "ProfileMap"
    WritePolicy wPolicy2 = new WritePolicy();
    MapPolicy mPolicy = new MapPolicy(MapOrder.KEY_ORDERED, MapWriteFlags.DEFAULT); 
    
    client.operate(wPolicy2, segKey, 
                   MapOperation.removeByKey("ProfileMap", Value.get(profileId), MapReturnType.NONE) 
                   );
   
    if(bCheck){
      System.out.println(client.get(null, segKey));
    }   
  }
    
  //For quick code dev testing, not used for Queries.  We wil use Batch get() for that. 
  public void getSegmentRecord(int segmentId, int profileId){
    int keyVal = segmentId;
    keyVal = keyVal << 8;
    int profileLSBits = profileId & 0xFF  ; // 0x 1111 1111
    keyVal = keyVal | profileLSBits;
    Key segKey = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal); 
    System.out.println(client.get(null, segKey));
  }
    
   //Alternate Data Model, store profile data in segment records 
   //This improves read operation efficieny. No need to do a second batch read of 
   //profile records after retrieving candidate profileIDs
   //but increases data stored in the database. 
   //Each segment will have the profile data of
   //each profile associated with it.  
   public void addProfileWithData (int segmentId, int profileId, int timeStamp, boolean bCheck) {
   
    int keyVal = segmentId;
    keyVal = keyVal << 8;
    int profileLSBits = profileId & 0xFF  ; // 0x 1111 1111
    keyVal = keyVal | profileLSBits;
   
    Key segKey = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal); 
    //Bin: In MapOperation, just pass bin name = "ProfileMap"
    WritePolicy wPolicy2 = new WritePolicy();
    MapPolicy mPolicy = new MapPolicy(MapOrder.KEY_ORDERED, MapWriteFlags.DEFAULT); 
       
    //Get Profile record data
    Key profileKey = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", profileId);
    Record rec = client.get(null, profileKey); 
    List<Value> profileData = new ArrayList<Value>();   
    profileData.clear();
    profileData.add(Value.get(timeStamp));
    profileData.add(Value.get(rec.getInt("ProfileId")));
    profileData.add(Value.get(rec.getString("Name")));
    profileData.add(Value.get(rec.getInt("Zip")));
       
    
    client.operate(wPolicy2, segKey, 
                   MapOperation.put(mPolicy,"ProfileMap", Value.get(profileId), Value.get(profileData)) 
                   );
   
    if(bCheck){
      System.out.println(client.get(null, segKey));
    }   
  }  
  
  //Queries - this for development test only. Does individual direct record read in a loop.  
  public void getAllSegmentRecords(int segmentId){
    int keyVal = 0;   
    for(int profileLSB=0; profileLSB<256; profileLSB++){
      keyVal = segmentId;
      keyVal = keyVal << 8;
      keyVal = keyVal | profileLSB;
      // segKey =  segmentID <<8 + LSB 0, 1, ....256  
      Key segKey = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal); 
      System.out.println(client.get(null, segKey));
    }
  }
  
  //Queries - this does the more efficient Batch read.
  public void batchGetAllSegmentProfileIDs(int segmentId){
    
    int keyVal = 0;   
    Key [] keys = new Key[256];    
    for(int profileLSB=0; profileLSB<256; profileLSB++){
      keyVal = segmentId;
      keyVal = keyVal << 8;
      keyVal = keyVal | profileLSB;
      keys[profileLSB] = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal);      
    }
    
    BatchPolicy bPolicy = new BatchPolicy();
    Record[] recs = new Record[256];
    Operation[] ops = new Operation[]{
       MapOperation.getByKeyRange("ProfileMap", Value.NULL, Value.INFINITY, MapReturnType.KEY)
       };
    recs = client.get(bPolicy, keys, ops);
    
    for(int i=0; i<256;i++){
      if(recs[i] != null) {
        System.out.println(recs[i]);
      }
    }
  }
    
  //If you want to return the Key & Data associated with MapKey .. 
    
  public void batchGetAllSegmentProfileData(int segmentId){
    
    int keyVal = 0;   
    Key [] keys = new Key[256];    
    for(int profileLSB=0; profileLSB<256; profileLSB++){
      keyVal = segmentId;
      keyVal = keyVal << 8;
      keyVal = keyVal | profileLSB;
      keys[profileLSB] = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal);      
    }
    
    BatchPolicy bPolicy = new BatchPolicy();
    Record[] recs = new Record[256];
    Operation[] ops = new Operation[]{
       MapOperation.getByKeyRange("ProfileMap", Value.NULL, Value.INFINITY, MapReturnType.VALUE)
       };
    recs = client.get(bPolicy, keys, ops);
    
    for(int i=0; i<256;i++){
      if(recs[i] != null) {
        System.out.println(recs[i]);
      }
    }
  }
  
  //This is for reading segmentID records when storing only ProfileIDs, by time range.
  //See next API for reading segmentID records when storing ProfileIDs with Profile Data, by time range.  
  public void batchGetSegmentProfileIDsByTS(int segmentId, int t0, int t1){
    
    int keyVal = 0;   
    Key [] keys = new Key[256];    
    for(int profileLSB=0; profileLSB<256; profileLSB++){
      keyVal = segmentId;
      keyVal = keyVal << 8;
      keyVal = keyVal | profileLSB;
      keys[profileLSB] = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal);      
    }
    
    BatchPolicy bPolicy = new BatchPolicy();
    Record[] recs = new Record[256];
    Operation[] ops = new Operation[]{
       MapOperation.getByValueRange("ProfileMap", Value.get(t0), Value.get(t1), MapReturnType.KEY)
       };  //Range is inclusive of t0 and exclusive of t1
    recs = client.get(bPolicy, keys, ops);
    
    for(int i=0; i<256;i++){
      if(recs[i] != null) {
        System.out.println(recs[i]);
      }
    }
  }
   
  //API for reading segmentID records when storing ProfileIDs with Profile Data, by time range.  
  public void batchGetSegmentProfileDataByTS(int segmentId, int t0, int t1){
    
    int keyVal = 0;   
    Key [] keys = new Key[256];    
    for(int profileLSB=0; profileLSB<256; profileLSB++){
      keyVal = segmentId;
      keyVal = keyVal << 8;
      keyVal = keyVal | profileLSB;
      keys[profileLSB] = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set2", keyVal);      
    }
    
    BatchPolicy bPolicy = new BatchPolicy();
    Record[] recs = new Record[256];
    
    List<Value> tsStart = new ArrayList<Value>();   
    tsStart.clear();
    tsStart.add(Value.get(t0));
    
    List<Value> tsEnd = new ArrayList<Value>();   
    tsEnd.clear();
    tsEnd.add(Value.get(t1));
    
    Operation[] ops = new Operation[]{
       MapOperation.getByValueRange("ProfileMap", 
                                    Value.ListValue.get(tsStart), Value.ListValue.get(tsEnd), 
                                    MapReturnType.KEY_VALUE)
       };  //Range is inclusive of t0 and exclusive of t1
    recs = client.get(bPolicy, keys, ops);
    
    for(int i=0; i<256;i++){
      if(recs[i] != null) {
        System.out.println(recs[i]);
      }
    }
}
  
}

In [8]:
//Create Profile records
class userProfile {
    
  segmentId pfSegId = new segmentId();  //Just run class segmentId cell first. figure out import later
    
  public void createProfile (int profileId, String name, int zip, boolean bCheck) {
      Key user = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", profileId); 
      Bin bProfileId = new Bin("ProfileId", profileId);
      Bin bName = new Bin("Name", name);
      Bin bZip = new Bin("Zip", zip);
      WritePolicy wPolicy1 = new WritePolicy();
      client.put(wPolicy1, user, bProfileId, bName, bZip);
      if(bCheck){
        System.out.println(client.get(null, user));
      }   
  }
    
  public void createProfileWithSegIdList(int profileId, String name, int zip, List<Value> segIds, int timeStamp, boolean bCheck) {

      Key user = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", profileId); 
      Bin bProfileId = new Bin("ProfileId", profileId);
      Bin bName = new Bin("Name", name);
      Bin bZip = new Bin("Zip", zip);
      //Bin bSegIds = new Bin("SegIds", segIds);
      WritePolicy wPolicy1 = new WritePolicy();
      ListPolicy lPolicy = new ListPolicy(ListOrder.ORDERED, ListWriteFlags.ADD_UNIQUE|ListWriteFlags.NO_FAIL|ListWriteFlags.PARTIAL);
      
      client.operate(wPolicy1, user, 
                     Operation.put(bProfileId), 
                     Operation.put(bName), 
                     Operation.put(bZip), 
                     ListOperation.appendItems(lPolicy, "SegIds", segIds)
                    );
      if(bCheck){
          System.out.println(client.get(null, user));
      }
      
      //Add profileId to Seg IDs
      //Signature: addProfile (int segmentId, int profileId, int timeStamp, boolean bCheck)
      //Use batch write ... later
      for(int i=0; i<segIds.size(); i++){
             pfSegId.addProfile(segIds.get(i).toInteger(), profileId, timeStamp, true);
          }
      
  } 
  
  public void updateProfileAppendSegIdList(int profileId, List<Value> segIds, int timeStamp, boolean bCheck) {
      Key user = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", profileId);       
      WritePolicy wPolicy1 = new WritePolicy();
   
      ListPolicy lPolicy = new ListPolicy(ListOrder.ORDERED, ListWriteFlags.ADD_UNIQUE|ListWriteFlags.NO_FAIL|ListWriteFlags.PARTIAL);
   
      Record rec = client.operate(wPolicy1, user,  
                     ListOperation.getByRankRange("SegIds", 0, ListReturnType.VALUE),
                     ListOperation.appendItems(lPolicy, "SegIds", segIds),
                     ListOperation.getByRankRange("SegIds", 0, ListReturnType.VALUE)
                    );
      
      //List<Value> initList = new ArrayList<Value>();
      List<?> retLists = rec.getList("SegIds");
      List<?> initList = (ArrayList<?>)retLists.get(0);      
      List<?> finalList = (ArrayList<?>)retLists.get(2);
      
      if(bCheck){
        System.out.println("Initial SegId List:"+ initList);
        System.out.println("SegId List to Append:"+finalList);
      }
      
      finalList.removeAll(initList);
      
      if(bCheck){
          System.out.println("New SegIds Appended, only update these:"+ finalList);
      }
      
      if(bCheck){
          System.out.println(client.get(null, user));
      }
      
      //Add profileId to Seg IDs
      //Signature: addProfile (int segmentId, int profileId, int timeStamp, boolean bCheck)
      //Use batch write ... later
      for(int i=0; i<finalList.size(); i++){
             pfSegId.addProfile(Math.toIntExact((Long)finalList.get(i)), profileId, timeStamp, true);
          }    
  } 
    
  public void updateProfileReplaceSegIdList(int profileId, List<Value> segIds, int timeStamp, boolean bCheck) {
      Key user = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", profileId);       
      WritePolicy wPolicy1 = new WritePolicy();
   
      ListPolicy lPolicy = new ListPolicy(ListOrder.ORDERED, ListWriteFlags.ADD_UNIQUE|ListWriteFlags.NO_FAIL|ListWriteFlags.PARTIAL);
  
      Record rec = client.operate(wPolicy1, user,  
                     ListOperation.getByRankRange("SegIds", 0, ListReturnType.VALUE),
                     ListOperation.clear("SegIds"),  //does not return any value by default
                     ListOperation.appendItems(lPolicy, "SegIds", segIds),
                     ListOperation.getByRankRange("SegIds", 0, ListReturnType.VALUE)
                    );
      
     
      List<?> retLists = rec.getList("SegIds");
      List<?> initList   = (ArrayList<?>)retLists.get(0); 
      List<?> deleteList = initList.stream().collect(Collectors.toList());  //Java 8
      List<?> finalList  = (ArrayList<?>)retLists.get(2);  //stays at 2 because clear() does not return a value      
      List<?> addList    = finalList.stream().collect(Collectors.toList());  //Java 8
      //List<?> addList    = List.copyOf(finalList);  //Java 10
      
      
      if(bCheck){
        System.out.println("Initial SegId List:"+ initList);
        System.out.println("Final SegId List:"+ finalList);
      }
      
    
      addList.removeAll(initList);      //All segIds not there initially
      
       
      if(bCheck){
          System.out.println("SegIds to Add List:"+ addList);
      }
      
      deleteList.removeAll(finalList);  //All segIds originally there but not in new list
      
      if(bCheck){          
          System.out.println("SegIds to Delete List:"+ deleteList);
      }
      
      if(bCheck){
          System.out.println(client.get(null, user));
      }
       
      //Add profileId to Seg IDs
      //Signature: addProfile (int segmentId, int profileId, int timeStamp, boolean bCheck)
      //Use batch write ... later
      for(int i=0; i<addList.size(); i++){
         pfSegId.addProfile(Math.toIntExact((Long)addList.get(i)), profileId, timeStamp, true);
      }
       
           //Use batch write ... later
      //Signature:  public void removeProfile (int segmentId, int profileId, boolean bCheck)
      for(int i=0; i<deleteList.size(); i++){
        pfSegId.removeProfile(Math.toIntExact((Long)deleteList.get(i)), profileId, true);
      }  
    
      
  } 
    
}

In [9]:
userProfile pf = new userProfile();
pf.createProfile(0,"Test", 94506, true);

(gen:1),(exp:0),(bins:(ProfileId:0),(Name:Test),(Zip:94506))


In [10]:
List<Value> lSegIds = new ArrayList<Value>();
lSegIds.add(Value.get(0xAA));
lSegIds.add(Value.get(0xBB));
lSegIds.add(Value.get(0xFF));

pf.createProfileWithSegIdList(1,"Test", 94506, lSegIds, 1234, true);

(gen:1),(exp:0),(bins:(ProfileId:1),(Name:Test),(Zip:94506),(SegIds:[170, 187, 255]))
SegmentId:0X000000AA
ProfileId:0X00000001
SegKey:0X0000AA01
SegKey:1010101000000001
(gen:1),(exp:0),(bins:(ProfileMap:{1=1234}))
SegmentId:0X000000BB
ProfileId:0X00000001
SegKey:0X0000BB01
SegKey:1011101100000001
(gen:1),(exp:0),(bins:(ProfileMap:{1=1234}))
SegmentId:0X000000FF
ProfileId:0X00000001
SegKey:0X0000FF01
SegKey:1111111100000001
(gen:1),(exp:0),(bins:(ProfileMap:{1=1234}))


In [11]:
List<Value> lSegIds = new ArrayList<Value>();
lSegIds.add(Value.get(0xCC));
lSegIds.add(Value.get(0xAA));
lSegIds.add(Value.get(0xBB));

pf.updateProfileAppendSegIdList(1, lSegIds, 1234, true);

Initial SegId List:[170, 187, 255]
SegId List to Append:[170, 187, 204, 255]
New SegIds Appended, only update these:[204]
(gen:2),(exp:0),(bins:(ProfileId:1),(Name:Test),(Zip:94506),(SegIds:[170, 187, 204, 255]))
SegmentId:0X000000CC
ProfileId:0X00000001
SegKey:0X0000CC01
SegKey:1100110000000001
(gen:1),(exp:0),(bins:(ProfileMap:{1=1234}))


In [12]:
List<Value> lSegIds = new ArrayList<Value>();
lSegIds.add(Value.get(0xDD));
lSegIds.add(Value.get(0xAA));
lSegIds.add(Value.get(0xBB));

pf.updateProfileReplaceSegIdList(1, lSegIds, 1234, true);

Initial SegId List:[170, 187, 204, 255]
Final SegId List:[170, 187, 221]
SegIds to Add List:[221]
SegIds to Delete List:[204, 255]
(gen:3),(exp:0),(bins:(ProfileId:1),(Name:Test),(Zip:94506),(SegIds:[170, 187, 221]))
SegmentId:0X000000DD
ProfileId:0X00000001
SegKey:0X0000DD01
SegKey:1101110100000001
(gen:1),(exp:0),(bins:(ProfileMap:{1=1234}))
SegmentId:0X000000CC
ProfileId:0X00000001
SegKey:0X0000CC01
SegKey:1100110000000001
(gen:2),(exp:0),(bins:(ProfileMap:{}))
SegmentId:0X000000FF
ProfileId:0X00000001
SegKey:0X0000FF01
SegKey:1111111100000001
(gen:2),(exp:0),(bins:(ProfileMap:{}))


In [13]:
segmentId segId = new segmentId();
segId.addProfile(1, 1, 1, false);
segId.addProfile(1, 2, 1, false);
segId.addProfile(1, 3, 1, false);
segId.addProfile(1, 4, 1, false);
segId.addProfile(1, 1025, 1, false);

// Test get call  (uses direct record reads) 
//Note: In this dev test, we are only adding a handful of Profile IDs 
//.. so most of the 1K records associated with this segmentID are null.
segId.getAllSegmentRecords(1);

null
(gen:2),(exp:0),(bins:(ProfileMap:{1=1, 1025=1}))
(gen:1),(exp:0),(bins:(ProfileMap:{2=1}))
(gen:1),(exp:0),(bins:(ProfileMap:{3=1}))
(gen:1),(exp:0),(bins:(ProfileMap:{4=1}))
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null

In [14]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, "set1", 0);
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, "set2", 0);

In [15]:
userProfile pf = new userProfile();
for (int i=0; i<10000; i++){
  pf.createProfile(i,"Name_"+i, 90000+i, false);
}

In [16]:
Key user = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", 269); 
System.out.println(client.get(null, user));

(gen:1),(exp:0),(bins:(ProfileId:269),(Name:Name_269),(Zip:90269))


In [17]:
segmentId segId = new segmentId();

Random rnd1 = new Random(0);  
//rnd1.nextInt(10) will give a random number between 0(incl.) and 10(excl.)

Random rnd2 = new Random(0);  

for(int i=0; i<10000; i++){  //Profile ID: 0 to 9,999

  int numSegs = rnd1.nextInt(10) + 1; 
  int rand_ts = rnd1.nextInt(10) + 1; 
  
  for(int j=0; j<numSegs; j++ ){  //Segment ID
    int ts = 1000 + (10*rand_ts);  // Different in each seg+prof record, ... just made up for testing.
    segId.addProfile(j, i, ts, false); 
    //segId.addProfileWithData(j, i, ts, false);  
  }
}


In [18]:
segId.getAllSegmentRecords(5);  // Not a batch get, run with 5, then edit to 6 and re-run.                 

(gen:23),(exp:0),(bins:(ProfileMap:{768=1010, 1280=1070, 1536=1100, 1792=1030, 2048=1030, 3072=1070, 3328=1100, 3584=1060, 4096=1020, 4608=1020, 5376=1060, 5888=1060, 6400=1020, 6912=1030, 7168=1060, 7424=1090, 7680=1020, 8192=1050, 8448=1100, 8704=1050, 8960=1060, 9472=1080, 9728=1020}))
(gen:26),(exp:0),(bins:(ProfileMap:{1=1080, 769=1020, 1281=1010, 1537=1040, 2561=1070, 2817=1030, 3329=1090, 3585=1090, 3841=1100, 4097=1060, 4353=1090, 4609=1060, 5121=1050, 5377=1040, 5633=1040, 5889=1090, 6145=1080, 6401=1040, 6657=1070, 7425=1050, 7681=1030, 7937=1060, 8193=1010, 8705=1080, 8961=1030, 9217=1030}))
(gen:14),(exp:0),(bins:(ProfileMap:{2=1040, 258=1010, 1026=1020, 1794=1040, 2306=1040, 2562=1070, 2818=1020, 3330=1030, 4354=1030, 4610=1020, 5378=1020, 6914=1010, 7682=1080, 9730=1030}))
(gen:15),(exp:0),(bins:(ProfileMap:{515=1010, 1027=1070, 2819=1020, 3075=1040, 3587=1090, 3843=1030, 4611=1050, 5123=1080, 5635=1100, 7171=1050, 7939=1080, 8707=1100, 9219=1040, 9475=1090, 9987=1030}))


In [19]:
segId.getSegmentRecord(5,4095);  
//Note: Must choose correct combination of seg Id and Profile id, e.g. one from 
//previous output for segment id = 5 to double check the record output.

(gen:18),(exp:0),(bins:(ProfileMap:{255=1070, 1535=1080, 1791=1060, 2303=1030, 2559=1020, 3583=1010, 4095=1080, 5119=1060, 5375=1040, 5887=1030, 6143=1080, 6911=1010, 7167=1040, 7423=1080, 7679=1090, 8191=1030, 8447=1100, 9727=1040}))


In [20]:
segId.batchGetAllSegmentProfileIDs(5);

(gen:23),(exp:0),(bins:(ProfileMap:[768, 1280, 1536, 1792, 2048, 3072, 3328, 3584, 4096, 4608, 5376, 5888, 6400, 6912, 7168, 7424, 7680, 8192, 8448, 8704, 8960, 9472, 9728]))
(gen:26),(exp:0),(bins:(ProfileMap:[1, 769, 1281, 1537, 2561, 2817, 3329, 3585, 3841, 4097, 4353, 4609, 5121, 5377, 5633, 5889, 6145, 6401, 6657, 7425, 7681, 7937, 8193, 8705, 8961, 9217]))
(gen:14),(exp:0),(bins:(ProfileMap:[2, 258, 1026, 1794, 2306, 2562, 2818, 3330, 4354, 4610, 5378, 6914, 7682, 9730]))
(gen:15),(exp:0),(bins:(ProfileMap:[515, 1027, 2819, 3075, 3587, 3843, 4611, 5123, 5635, 7171, 7939, 8707, 9219, 9475, 9987]))
(gen:17),(exp:0),(bins:(ProfileMap:[4, 260, 1796, 2052, 2308, 2564, 2820, 3076, 3588, 4868, 6148, 6404, 7940, 8452, 8708, 9732, 9988]))
(gen:26),(exp:0),(bins:(ProfileMap:[5, 517, 1029, 2309, 2565, 2821, 3077, 3333, 3589, 3845, 4101, 4357, 4869, 5125, 5381, 5893, 6405, 6661, 7173, 7429, 7685, 8197, 8965, 9221, 9733, 9989]))
(gen:20),(exp:0),(bins:(ProfileMap:[262, 1286, 2054, 2822, 3078,

In [21]:
Key user = new Key(AEROSPIKE_CLOUD_NAMESPACE, "set1", 4095); 
System.out.println(client.get(null, user));

(gen:1),(exp:0),(bins:(ProfileId:4095),(Name:Name_4095),(Zip:94095))


In [22]:
segId.batchGetAllSegmentProfileIDs(5);
segId.batchGetAllSegmentProfileIDs(0);
segId.batchGetAllSegmentProfileIDs(9);

(gen:23),(exp:0),(bins:(ProfileMap:[768, 1280, 1536, 1792, 2048, 3072, 3328, 3584, 4096, 4608, 5376, 5888, 6400, 6912, 7168, 7424, 7680, 8192, 8448, 8704, 8960, 9472, 9728]))
(gen:26),(exp:0),(bins:(ProfileMap:[1, 769, 1281, 1537, 2561, 2817, 3329, 3585, 3841, 4097, 4353, 4609, 5121, 5377, 5633, 5889, 6145, 6401, 6657, 7425, 7681, 7937, 8193, 8705, 8961, 9217]))
(gen:14),(exp:0),(bins:(ProfileMap:[2, 258, 1026, 1794, 2306, 2562, 2818, 3330, 4354, 4610, 5378, 6914, 7682, 9730]))
(gen:15),(exp:0),(bins:(ProfileMap:[515, 1027, 2819, 3075, 3587, 3843, 4611, 5123, 5635, 7171, 7939, 8707, 9219, 9475, 9987]))
(gen:17),(exp:0),(bins:(ProfileMap:[4, 260, 1796, 2052, 2308, 2564, 2820, 3076, 3588, 4868, 6148, 6404, 7940, 8452, 8708, 9732, 9988]))
(gen:26),(exp:0),(bins:(ProfileMap:[5, 517, 1029, 2309, 2565, 2821, 3077, 3333, 3589, 3845, 4101, 4357, 4869, 5125, 5381, 5893, 6405, 6661, 7173, 7429, 7685, 8197, 8965, 9221, 9733, 9989]))
(gen:20),(exp:0),(bins:(ProfileMap:[262, 1286, 2054, 2822, 3078,

In [23]:
segId.batchGetSegmentProfileIDsByTS(5, 1020, 1050);  //valueEnd exclusive, try 1051 to include 1050

(gen:23),(exp:0),(bins:(ProfileMap:[1792, 2048, 4096, 4608, 6400, 6912, 7680, 9728]))
(gen:26),(exp:0),(bins:(ProfileMap:[769, 1537, 2817, 5377, 5633, 6401, 7681, 8961, 9217]))
(gen:14),(exp:0),(bins:(ProfileMap:[2, 1026, 1794, 2306, 2818, 3330, 4354, 4610, 5378, 9730]))
(gen:15),(exp:0),(bins:(ProfileMap:[2819, 3075, 3843, 9219, 9987]))
(gen:17),(exp:0),(bins:(ProfileMap:[1796, 2564, 3588, 6148, 7940, 8452]))
(gen:26),(exp:0),(bins:(ProfileMap:[1029, 2309, 2565, 3589, 3845, 4101, 4357, 4869, 5125, 5381, 5893, 6661, 7685, 8197, 8965, 9989]))
(gen:20),(exp:0),(bins:(ProfileMap:[2054, 2822, 3846, 7174, 7686, 8710]))
(gen:23),(exp:0),(bins:(ProfileMap:[263, 2055, 4103, 6919, 8711]))
(gen:21),(exp:0),(bins:(ProfileMap:[520, 5896]))
(gen:18),(exp:0),(bins:(ProfileMap:[521, 1289, 1801, 2569, 3081, 5385]))
(gen:19),(exp:0),(bins:(ProfileMap:[1802, 3338, 5898, 8202, 9482]))
(gen:20),(exp:0),(bins:(ProfileMap:[1035, 1803, 2315, 3339, 5131, 5643]))
(gen:26),(exp:0),(bins:(ProfileMap:[780, 1036, 

In [24]:
client.close();